In [ ]:
def get_google_results(navegador, max_candidates):
    # Determina o número de páginas a serem processadas
    max_candidates = round(max_candidates / 5)
    perfis = []  # Lista para armazenar todos os perfis

    for i in range(max_candidates):
        # Busca elementos na página atual
        google_results = navegador.driver.find_elements(By.XPATH, '//span[@jscontroller="msmzHf"]')
        print(f"Processando candidato {i+1} de {max_candidates}")

        for result in google_results:
            
            try:
                # Tentando obter o link
                try:
                    link = result.find_element(By.TAG_NAME, "a").get_attribute("href")

                    # Normaliza o prefixo para remover qualquer idioma ou região desnecessários
                    for prefix in ["/pt", "/en", "/es", "/fr", "/de"]:  # Adicione outros idiomas, se necessário
                        link = link.replace(prefix, "")

                    # Remove o prefixo "https://br." para uniformizar os links
                    link = link.replace("https://br.", "https://")

                    # Remove qualquer prefixo antes de linkedin.com (cm., ke., etc.)
                    link = link.split('linkedin.com', 1)[-1]  # Mantém a parte após 'linkedin.com'
                    link = "https://linkedin.com" + link  # Adiciona o prefixo padrão 'https://linkedin.com'

                except:
                    link = "Link não encontrado"


                # Adicionando os dados à lista de perfis
                perfis.append({
                    "link": link,
                    "nome": "",
                    "skills": "",
                    "sobre": "",
                    "cargo": "",
                    "experiencia": [],
                    "educacao": [],
                    "certificacoes": [],
                    "contato": {
                        "email": "",
                        "telefone": "",
                        "linkedin": "",
                        "github": ""
                    }
                })
            except Exception as e:
                print(f"Erro ao processar candidato: {e}")
        
        # Avança para a próxima página
        navegador.click("ID", "pnnext")

    return perfis


In [ ]:
def update_linkedin_profile(linkedin_profile, update_profile=False):

    import dotenv
    import sqlite3
    dotenv.load_dotenv()

    cookie = {
        "name": "li_at",
        "value": os.environ["LINKEDIN_VISITOR_ID"],
        "domain": ".linkedin.com"
    }

    navegador = Navegador()

    navegador.get('https://www.linkedin.com/feed/')

    navegador.driver.add_cookie(cookie) 

    linkedin_prof = linkedin_profile

    for profile in linkedin_prof:

        #verifica se o perfil ja existe no banco de dados
        conn = sqlite3.connect("profiles.db")
        cursor = conn.cursor()
        cursor.execute("SELECT link FROM profile WHERE link = ?", (profile['link'],))
        if cursor.fetchone() is not None and update_profile == False:
            continue

        if profile['link'] == "Link não encontrado":
            continue
        print(f"Coletando dados do perfil numero {linkedin_prof.index(profile)} do total de {len(linkedin_prof)}")
        profile_url = profile['link']

        #carregar o perfil
        navegador.get(profile_url)
        
        if not is_page_exists(navegador):
            #pula o perfil
            continue

        if is_page_is_not_found(navegador):
            #pula o perfil
            continue

        soup = BeautifulSoup(navegador.driver.page_source, 'html.parser')
        #scrola a pagina e aguarda a pagina carregar
        navegador.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        try:
            # Aumentando o tempo de espera para 20 segundos
            name_element = WebDriverWait(navegador.driver, 20).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, 'h1.inline.t-24.v-align-middle.break-words'))
            )
            name = name_element.text.strip() if name_element else "N/A"
        except TimeoutException:
            print("Nome não encontrado. Pode ser uma página de erro.")
            name = "N/A"

        # Verifique se name_element é um objeto BeautifulSoup e use get_text apenas se não for None
        

        try:

            photo_element = soup.select_one('img.pv-top-card-profile-picture__image--show.evi-image')
            photo_url = photo_element.get('src') if photo_element else "N/A"

            headline = soup.find('div', {'class': 'text-body-medium break-words'})
            headline = headline.get_text().strip()

            about = soup.find('div', {'class': 'display-flex ph5 pv3'}) if soup.find('div', {'class': 'display-flex ph5 pv3'}) else "N/A"
        except TimeoutException:
            headline = "N/A"
            about = "N/A"
            photo_url = "N/A"
        if about == "N/A":
            about = "N/A"
        else:
            about = about.get_text().strip()

        #atualiza o perfil
        profile['nome'] = name
        profile['skills'] = headline
        profile['sobre'] = about
        profile['foto'] = photo_url
        #coleta de experiencia
        print('-'*100)
        print(f"coletando experiencia, educacao e certificacoes de {name}")
        experiences = get_experiences(navegador, profile_url)
        profile['experiencia'] = experiences
        #coleta de educacao
        education = get_education(navegador, profile_url)
        profile['educacao'] = education
        #coleta de certificacoes
        certifications = get_certifications(navegador, profile_url)
        profile['certificacoes'] = certifications
        
        print(f"dados de {name} coletados com sucesso")
        print('-'*100)

        salvar_perfil_unico(profile)

    navegador.close()
    
    return linkedin_prof


In [49]:
import requests

response = requests.post(
    "http://127.0.0.1:3000//searchprofiles",  # Substitua pelo endpoint correto
    json={
        "cargos": ["Desenvolvedor Front End"],
        "habilidades": ["Django", "FastAPI"],
        "ferramentas": ["Docker", "Kubernetes"],
        "localizacoes": ["Brasil", "Portugal"],
        "max_interactions": 5,
        "job_bubble_id": "1738117388263x970142495908560900",
    },
    headers={"Content-Type": "application/json"}
)

print(response.json())


{'message': 'Pesquisa realizada com sucesso', 'result': {'status': 'Busca realizada com sucesso'}}


In [51]:
import requests

BASE_URL = "http://127.0.0.1:3000"  # Altere para o IP correto se necessário

# Dados a serem enviados
json_data = {
    "job_id": "aaaaaa",
    "max_candidates": 5,
    "cargos": ["Desenvolvedor Front End"],
    "habilidades": ["Django", "FastAPI"],
    "ferramentas": ["Docker", "Kubernetes"],
    "localizacoes": ["Brasil", "Portugal"],
    "max_interactions": 5,
    "job_bubble_id": "1738117388263x970142495908560900"
}

# Fazendo a requisição POST
response = requests.post(f"{BASE_URL}/add_job", json=json_data)

# Exibindo a resposta
print("Status Code:", response.status_code)
print("Response JSON:", response.json())


Status Code: 201
Response JSON: {'inserted_id': 1, 'job_id': 'aaaaaa', 'message': 'Job adicionado'}


In [45]:
import requests

BASE_URL = "http://127.0.0.1:3000"  # Altere para o IP correto se necessário

# Dados a serem enviados
data = {
    "job_id": "job_123",
    "status": 1
}

update_job_status("job_123", 0)

Job job_123 atualizado para status 0
